# Interoperability with Atomic Simulation Environment

\[[View with IPython nbviewer](http://nbviewer.ipython.org/github/libAtoms/QUIP/blob/public/doc/Tutorials/quippy-ase-interoperability.ipynb)\]

- quippy now uses the standard `ase.atoms.Atoms` class to represent Atomic structures
- quippy `Potential` objects can be used as ASE calculators, and vice-versa
- Can use standard ASE tools, plus communicate with other packages using ASE as *lingua franca*

## Example: vacancy formation energy

- Generate structure with `ASE` lattice tools
- Stillinger-Weber potential implementation from `QUIP`
- Elastic constant fitting routine from `matscipy`, internal relaxations with `ASE` FIRE minimiser

In [7]:
%pylab inline
from ase.build import bulk
from ase.optimize import BFGS
from ase.optimize.precon import PreconLBFGS
from quippy.potential import Potential

si = bulk('Si', a=5.44, cubic=True)
sw_pot = Potential('IP SW', 
                   param_filename='../../share/Parameters/ip.parms.SW.xml')
si.set_calculator(sw_pot)
e_bulk_per_atom = si.get_potential_energy()/len(si)

# call general purpose elastic constants calculator 
#   using ASE Atoms and QUIP Potential
from matscipy.elasticity import fit_elastic_constants
Cij = fit_elastic_constants(si, optimizer=BFGS,
                            symmetry='cubic', logfile='-')
vac1 = si.copy()
vac1 *= (3, 3, 3)
half_cell = np.diag(vac1.cell)/2.
vac_atom = ((vac1.positions - half_cell)**2).sum(axis=1).argmin()
del vac1[vac_atom]

vac1.set_calculator(sw_pot)
vac1.rattle(0.01)
opt = PreconLBFGS(vac1) # big cell, use preconditioned minimiser
opt.run(fmax=1e-6)
e_vac = vac1.get_potential_energy() - e_bulk_per_atom*len(vac1)
print('SW vacancy formation energy', e_vac, 'eV')

Populating the interactive namespace from numpy and matplotlib
      Step     Time          Energy         fmax
BFGS:    0 21:22:10      -34.635777        0.3247
BFGS:    1 21:22:10      -34.644590        0.1504
BFGS:    2 21:22:10      -34.646997        0.0001
      Step     Time          Energy         fmax
BFGS:    0 21:22:10      -34.670667        0.1584
BFGS:    1 21:22:10      -34.672777        0.0749
BFGS:    2 21:22:10      -34.673385        0.0000
      Step     Time          Energy         fmax
BFGS:    0 21:22:10      -34.678737        0.0000
      Step     Time          Energy         fmax
BFGS:    0 21:22:10      -34.660845        0.1508
BFGS:    1 21:22:10      -34.662784        0.0742
BFGS:    2 21:22:10      -34.663403        0.0000
      Step     Time          Energy         fmax
BFGS:    0 21:22:10      -34.617822        0.2945
BFGS:    1 21:22:10      -34.625262        0.1477
BFGS:    2 21:22:10      -34.627763        0.0001
Fitting C_11
Strain array([-0.02, -0.01,  